In [10]:
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoGeo")
import AutoGeo

# Previous

In [10]:
import os
import re
import numpy as np
import rasterio
from datetime import datetime, timedelta

# 假设文件夹路径为 'modis_data_folder'
folder_path = r'C:\Users\isxzl\OneDrive\Code\AutoGeo\data\AOD\MOD'
folder_path = r'C:\Users\isxzl\OneDrive\Code\AutoGeo\data\NDVI'

def retrieve_tiff_files(folder, date_range):
    # 日期范围处理
    start_date_str, end_date_str = date_range
    start_date = datetime.strptime(start_date_str, '%Y%j')
    end_date = datetime.strptime(end_date_str, '%Y%j')

    # 使用正则表达式从文件名中提取日期（假设格式为 'YYYYDDD'）
    date_pattern = re.compile(r'\d{7}')

    # 找到所有的TIFF文件
    tiff_files = [f for f in os.listdir(folder) if (f.endswith('.tiff') or f.endswith('.tif')) and date_pattern.search(f)]

    # 解析日期，并转换为datetime对象
    dates = [datetime.strptime(date_pattern.search(f).group(), '%Y%j') for f in tiff_files]
    dates = [d for d in dates if start_date <= d <= end_date]
    dates.sort()

    # 计算缺失日期
    all_dates = {start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)}
    missing_dates = all_dates - set(dates)

    return [f for f in tiff_files if start_date <= datetime.strptime(date_pattern.search(f).group(), '%Y%j') <= end_date], date_range, missing_dates


# 步骤 2: 打印第一个TIFF文件的所有meta data
def print_first_tiff_metadata(folder, tiff_files):
    first_tiff = tiff_files[0]
    with rasterio.open(os.path.join(folder, first_tiff)) as src:
        print(src.meta)

# 步骤 3: 创建一个三维numpy矩阵
def create_3d_matrix(folder, tiff_files):
    # 假设所有TIFF文件具有相同的尺寸和分辨率
    with rasterio.open(os.path.join(folder, tiff_files[0])) as src:
        # 获取单个TIFF文件的尺寸
        height, width = src.shape

    # 初始化三维矩阵：(时间, 高度, 宽度)
    time_length = len(tiff_files)
    data_matrix = np.empty((time_length, height, width))

    # 加载每个TIFF文件的数据
    for i, tiff_file in enumerate(tiff_files):
        with rasterio.open(os.path.join(folder, tiff_file)) as src:
            data_matrix[i, :, :] = src.read(1)  # 假设数据在第一波段

    return data_matrix





In [11]:
# 执行以上函数
tiff_files, date_range, missing_dates = retrieve_tiff_files(folder_path,["2022001","2022015"])
print_first_tiff_metadata(folder_path, tiff_files)
data_matrix = create_3d_matrix(folder_path, tiff_files)
# 打印结果
date_range, missing_dates
data_matrix.shape

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': None, 'width': 1085, 'height': 552, 'count': 1, 'crs': CRS.from_wkt('PROJCS["WGS 1984. Custom Ellipsoid: Semi-major axis=6371007.181000, Semi-minor axis=6371007.181000.;scale=10000.00000000000000000000,offset=0.00000000000000000000",GEOGCS["World Geodetic System 1984",DATUM["WGS_1984",SPHEROID["WGS 84",6371007.181,0,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Sinusoidal"],PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(927.053216133641, 0.0, 11223248.257367,
       0.0, -926.3922659782609, 3469285.621672)}


(1, 552, 1085)

In [29]:
import rasterio
import pyproj
from rasterio.transform import from_origin

def latlon_to_raster_coords(folder_path, tiff, lat, lon):
    # 打开栅格数据集
    with rasterio.open(os.path.join(folder_path, tiff)) as tiff:
        dataset_crs = tiff.crs
        transform = tiff.transform
    
        # 创建一个转换器，将WGS 84 (经纬度) 转换为数据集的CRS
        transformer = pyproj.Transformer.from_crs("epsg:4326", dataset_crs, always_xy=True)
        
        # 转换经纬度坐标
        x, y = transformer.transform(lon, lat)
    
        return x, y 

# 示例使用
tiff=tiff_files[0]
lat = 23   # 替换为您的纬度
lon = -160   # 替换为您的经度
row, col = latlon_to_raster_coords(folder_path, tiff, lat, lon)

with rasterio.open(os.path.join(folder_path, tiff)) as tiff:
    dataset_crs = tiff.crs

tiff.crs

CRS.from_wkt('PROJCS["WGS 1984. Custom Ellipsoid: Semi-major axis=6371007.181000, Semi-minor axis=6371007.181000.;scale=10000.00000000000000000000,offset=0.00000000000000000000",GEOGCS["World Geodetic System 1984",DATUM["WGS_1984",SPHEROID["WGS 84",6371007.181,0,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Sinusoidal"],PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')

In [13]:
tiff_files[0]

'MOD13A2.A2022001.mosaic.061.2024007132737.psmcgs_000502078953.1_km_16_days_NDVI-1_km_16_days_NDVI.tif'

Prompts

编写一个函数，给定字符串，可以提取出对应的日期
给定%yyyy%ddd  #年份+儒略日
给定%yyyy%mm%dd #年份+月+日


然后利用这个函数，给定1个时间
time0

给定一个时间list, time1, time2, time3
查找和time0最近的time



rasterfilepath= 
date_formats= [format1, format2,.]
times=[time1, time2,.]，清洗

2. 根据时间，查找最近的时间的文件列表

3. 获取对应的像素值

print(pixe

给定以上函数，
给定变量coordinates=[[coordinate1,datetime(object)], [coordinate2,datetime2(object)], etc,.]
crs="epsg:4326"和已经排序好的变量sorted_tiff_lists 


1. （将coordinates转换为pandas dataframe:  列包括coordinate， datetime，传入的sorted_tiff_lists 的 所有key 每个key成为一个column），根据coordinates存储 coordinate， datetime的值，对于每个key column 创建一个空字典 {relative_coordinate, filepath, value, time_distance, space_distance}
2. 接下来我们要存储key column的值，对于每行(each coordinate)，每个key column：使用coordinates和sorted_tiff_lists 中的crs和transform信息 利用latlon_to_raster_coords函数获得对应的相对坐标,存储在relative_coordinate对应的值中，然后查找其最近的日期对应的filepath存储, 将日期的差值(小时)存储到pandas的time_distance中。
3. 之后，对于每个coordinate，根据获得的相对坐标和filepath，读取其像元值，存储到value中，确保所有文件只会读取一次，所以你需要整理好读取的顺序，优化算法s)


In [33]:
import glob
import pandas as pd
from datetime import datetime, timedelta, date

import re
import rasterio
import pyproj
from rasterio.transform import from_origin
import os
def latlon_to_raster_coords(crs_source,crs_target, transform, lat, lon):
        # 创建一个转换器，将WGS 84 (经纬度) 转换为数据集的CRS
        transformer = pyproj.Transformer.from_crs(crs_source, crs_target, always_xy=True)
        # 转换经纬度坐标
        x, y = transformer.transform(lon, lat)
    
        return x, y 
def extract_date_from_string(str1, flag, date_format):
    """
    Extracts a date from a string based on the provided flag and date format.
    # Example usage
    str1 = "MYD04_L2.A2022001.mosaic.061.2024007025724.pssggs_000502078783.Deep_Blue_Aerosol_Optical_Depth_550_Land-Deep_Blue_Aerosol_Optical_Depth_550_Land"
    flag = ["L2.A", ".mosaic"]
    date_format = "%yyyy%ddd"
    extract_date_from_string(str1,flag,date_format)
    """
    # Extract the date part from the string using the flag
    start_flag, end_flag = flag
    start_index = str1.find(start_flag) + len(start_flag)
    end_index = str1.find(end_flag, start_index)
    date_str = str1[start_index:end_index]

    # Parse the date based on the given format
    try:
        if date_format == '%yyyy%ddd':
            return datetime.strptime(date_str, '%Y%j').date()
        elif date_format == '%yyyy%mm%dd':
            return datetime.strptime(date_str, '%Y%m%d').date()
    except ValueError:
        return None

    return None



def get_sorted_tiff_list_by_date(rasterfilepaths):

    """
    Processes a dictionary of raster file information and returns a modified dictionary with an additional key 'filelistbydate'. 
    This key holds a pandas DataFrame of TIFF files sorted by their extracted dates.

    Parameters:
    rasterfilepaths (dict): A dictionary where each key is a custom label for a set of raster files. 
                            Each value is another dictionary with the following keys:
                              - 'path': A string representing the directory path containing TIFF files.
                              - 'flag': A tuple of two strings, where the first string is the start flag and the second string is the end flag. 
                                        These flags are used to extract the date portion from the TIFF file names.
                              - 'dateformat': A string representing the date format. Supported formats are '%yyyy%ddd' and '%yyyy%mm%dd'.

    Returns:
    dict: A dictionary similar to the input but with an additional key 'filelistbydate' for each original key. 
          The value of 'filelistbydate' is a pandas DataFrame containing the sorted list of TIFF files and their extracted dates.

    # Example usage
    rasterfilepaths = {
        "NDVI": {
            "path": "C:/Users/isxzl/OneDrive/Code/AutoGeo/data/NDVI",
            "flag": ["A2.A", ".mosaic"],
            "dateformat": "%yyyy%ddd"
        },
        "AOD:MOD": {
            "path": "C:/Users/isxzl/OneDrive/Code/AutoGeo/data/AOD/MOD",
            "flag": ["L2.A", ".mosaic"],
            "dateformat": "%yyyy%ddd"
        }
    }
    
    sorted_tiff_lists = get_sorted_tiff_list_by_date(rasterfilepaths)
    sorted_tiff_lists

    Note:
    - This function requires the 'glob' and 'pandas' modules.
    - The function does not execute file system operations in this environment. It should be run in a local environment where file system access is available.
    """
 
    sorted_filelists = {}

    for key, value in rasterfilepaths.items():
        path = value['path']
        flag = value['flag']
        date_format = value['dateformat']

        # Use glob to get the list of tiff files
        tiff_files = glob.glob(f"{path}/*.tif")

        # Extract dates and create a DataFrame
        file_dates = []
        for file in tiff_files:
            filename = file.split('/')[-1]
            date = extract_date_from_string(filename, flag, date_format)
            if date:
                file_dates.append([file, date])
            else:
                raise ValueError('Do not detect date for file'+str(filename))

        df = pd.DataFrame(file_dates, columns=['Filename', 'Date'])
        # Sort DataFrame by date
        df_sorted = df.sort_values(by='Date')

        # Add the sorted DataFrame to the dictionary
        sorted_filelists[key] = value

        with rasterio.open(file) as tiff:
            dataset_crs = tiff.crs
            transform = tiff.transform
            sorted_filelists[key]['crs']=dataset_crs
            sorted_filelists[key]['transform']=transform
        sorted_filelists[key]['filelistbydate'] = df_sorted
        
    return sorted_filelists



# 新函数：将坐标和时间转换为DataFrame
def create_dataframe(coordinates, sorted_tiff_lists):
    df = pd.DataFrame(coordinates, columns=['Coordinate', 'Datetime'])
    for key in sorted_tiff_lists.keys():
        df[key] = None
    return df

# 新函数：处理DataFrame
def process_dataframe(df, sorted_tiff_lists, crs_source):
    for i, row in df.iterrows():
        for key in sorted_tiff_lists.keys():
            dataset_info = sorted_tiff_lists[key]
            crs_target = dataset_info['crs']
            transform = dataset_info['transform']
            lat, lon = row['Coordinate']

            # 计算相对坐标
            relative_coord = latlon_to_raster_coords(crs_source, crs_target, transform, lat, lon)

            # 找出最接近的时间点的文件
            closest_file, time_diff = find_closest_file(row['Datetime'], dataset_info['filelistbydate'])

            # 存储信息
            df.at[i, key] = {'relative_coordinate': relative_coord, 'time_distance': time_diff, 'filepath': closest_file}

    return df


def find_closest_file(target_datetime, df_sorted):
    # 确保目标日期是datetime类型
    if isinstance(target_datetime, pd.Timestamp):
        target_datetime = target_datetime.to_pydatetime()

    # 转换df_sorted中的日期为datetime类型，如果需要的话
    df_sorted['Date'] = df_sorted['Date'].apply(lambda x: datetime.combine(x, datetime.min.time()) 
                                                if isinstance(x, date) and not isinstance(x, datetime) 
                                                else x)

    # 二分查找最接近的日期
    min_index = 0
    max_index = len(df_sorted) - 1
    closest_index = None
    min_time_diff = float('inf')

    while min_index <= max_index:
        mid_index = (min_index + max_index) // 2
        mid_date = df_sorted.iloc[mid_index]['Date']

        time_diff = abs((mid_date - target_datetime).total_seconds())
        
        if time_diff < min_time_diff:
            min_time_diff = time_diff
            closest_index = mid_index

        if mid_date < target_datetime:
            min_index = mid_index + 1
        elif mid_date > target_datetime:
            max_index = mid_index - 1
        else:
            break  # 直接命中

    closest_file = df_sorted.iloc[closest_index]['Filename']
    time_diff_hours = min_time_diff / 3600

    return closest_file, time_diff_hours



    
# Example usage
rasterfilepaths = {
    "NDVI": {
        "path": "C:/Users/isxzl/OneDrive/Code/AutoGeo/data/NDVI",
        "flag": ["A2.A", ".mosaic"],
        "dateformat": "%yyyy%ddd"
    },
    "AOD:MOD": {
        "path": "C:/Users/isxzl/OneDrive/Code/AutoGeo/data/AOD/MOD",
        "flag": ["L2.A", ".mosaic"],
        "dateformat": "%yyyy%ddd"
    },
    "AOD:MYD": {
        "path": "C:/Users/isxzl/OneDrive/Code/AutoGeo/data/AOD/MYD",
        "flag": ["L2.A", ".mosaic"],
        "dateformat": "%yyyy%ddd"
        }
}

sorted_tiff_lists = get_sorted_tiff_list_by_date(rasterfilepaths)

# 示例使用  118, 26.60, 123, 31.20
coordinates = [[(28.6895, 119.6917), datetime(2022, 1, 10)], [(28.7128, 121.0060), datetime(2022, 1, 15)]]
crs_source = "epsg:4326"

# 假设sorted_tiff_lists已经按照您的函数生成
sorted_tiff_lists = get_sorted_tiff_list_by_date(rasterfilepaths)

df = create_dataframe(coordinates, sorted_tiff_lists)
processed_df = process_dataframe(df, sorted_tiff_lists, crs_source)

# 打印处理后的DataFrame
#print(processed_df)

processed_df




,Coordinate,Datetime,NDVI,AOD:MOD,AOD:MYD
0,"(28.6895, 119.6917)",2022-01-10,"{'relative_coordinate': (11675218.843786215, 3...","{'relative_coordinate': (119.6917, 28.6895), '...","{'relative_coordinate': (119.6917, 28.6895), '..."
1,"(28.7128, 121.006)",2022-01-15,"{'relative_coordinate': (11800793.295348419, 3...","{'relative_coordinate': (121.006, 28.7128), 't...","{'relative_coordinate': (121.006, 28.7128), 't..."


In [34]:
def organize_file_reads(df, sorted_tiff_lists):
    """
    整理文件读取顺序和相关坐标。

    Parameters:
    df (pandas.DataFrame): 包含坐标和文件信息的DataFrame。

    Returns:
    dict: 键为文件路径，值为坐标列表的字典。
    """
    file_reads = {}
    for i, row in df.iterrows():
        for key in sorted_tiff_lists.keys():
            if row[key] is not None:
                info = row[key]
                filepath = info['filepath']
                relative_coord = info['relative_coordinate']

                if filepath not in file_reads:
                    file_reads[filepath] = []
                file_reads[filepath].append((i, key, relative_coord))

    return file_reads

def batch_read_pixel_values(file_reads):
    """
    批量读取像素值。

    Parameters:
    file_reads (dict): 需要读取的文件和坐标的字典。

    Returns:
    dict: 包含像素值的字典，键为元组 (行索引, 数据集键), 值为像素值。
    """
    pixel_values = {}
    for filepath, coords in file_reads.items():
        with rasterio.open(filepath) as src:
            for i, key, relative_coord in coords:
                pixel_value = next(src.sample([relative_coord]))[0]  # 读取第一个波段的值
                pixel_values[(i, key)] = pixel_value

    return pixel_values

def update_dataframe_with_values(df, pixel_values):
    """
    使用读取到的像素值更新DataFrame。

    Parameters:
    df (pandas.DataFrame): 包含坐标和文件信息的DataFrame。
    pixel_values (dict): 包含像素值的字典。

    Returns:
    pandas.DataFrame: 更新后的DataFrame。
    """
    for (i, key), value in pixel_values.items():
        info = df.at[i, key]
        info['value'] = value
        df.at[i, key] = info

    return df

# 整理文件读取顺序
file_reads = organize_file_reads(processed_df, sorted_tiff_lists)

# 批量读取像素值
pixel_values = batch_read_pixel_values(file_reads)

# 更新DataFrame
final_df = update_dataframe_with_values(processed_df, pixel_values)

# 打印最终的DataFrame
print(final_df)


            Coordinate   Datetime  \
0  (28.6895, 119.6917) 2022-01-10   
1   (28.7128, 121.006) 2022-01-15   

                                                NDVI  \
0  {'relative_coordinate': (11675218.843786215, 3...   
1  {'relative_coordinate': (11800793.295348419, 3...   

                                             AOD:MOD  \
0  {'relative_coordinate': (119.6917, 28.6895), '...   
1  {'relative_coordinate': (121.006, 28.7128), 't...   

                                             AOD:MYD  
0  {'relative_coordinate': (119.6917, 28.6895), '...  
1  {'relative_coordinate': (121.006, 28.7128), 't...  


{'relative_coordinate': (11800793.295348419, 3192721.288395223),
 'time_distance': 336.0,
 'filepath': 'C:/Users/isxzl/OneDrive/Code/AutoGeo/data/NDVI\\MOD13A2.A2022001.mosaic.061.2024007132737.psmcgs_000502078953.1_km_16_days_NDVI-1_km_16_days_NDVI.tif',
 'value': 7955}

# Load_SRTM

given:
- SRTM_dir

wanted:
- Xarray contains all data.

In [1]:
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoGeo")
from sample.geotiff import load_srtm
import rioxarray

srtm_path=r"C:\Datasets\Zhejiang20-23RS\DEM_SRTM/2022001.img"
ds=load_srtm(srtm_path)



Current CRS: EPSG:4326
